# 캐치 크롤링
- 크롤링할 컬럼: 기업명, 기업형태, 매출액, 사원수, 업종분류, 설립연도
- 절차
1. url로 기업명 입력
2. 기업정보 사이트 접속
3. 내용 크롤링
4. concat
5. 다음 url 입력

위 절차를 반복합니다.

In [ ]:
import pandas as pd
# 정적크롤링
from bs4 import BeautifulSoup
import requests
import re
# 동적크롤링
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [ ]:
# 파일 불러오기: 불러올 파일을 현재 폴더에 넣어두세요.
df = pd.read_csv('./programmers.csv')  # 파일명만 변경하여 그대로 돌리시면 됩니다.
df_corp_name = df['기업명']  # 기업명 불러오기

In [ ]:
# 임시 프레임
catch_res = pd.DataFrame(columns=['기업명','기업형태','매출액','사원수','업종분류','설립연도'])

In [ ]:
# 크롤링 프로그램
for name in df_corp_name:
    try:
        url = f'https://www.catch.co.kr/Search/SearchDetail?Menu=2&Keyword={name}'

        driver= webdriver.Chrome()
        driver.get(url)

        driver.find_element(By.XPATH, '//*[@id="leftTop"]/ul/li/div[2]/p[1]/a').click()
        time.sleep(0.2)

        # 기업명
        corp_name = name
        # 기업형태
        corp_type = driver.find_element(By.XPATH, '//*[@id="Contents"]/div[2]/div[1]/table/tbody/tr[2]/td[1]').text
        # 매출액
        sales_money = driver.find_element(By.XPATH, '//*[@id="Contents"]/div[2]/div[1]/table/tbody/tr[3]/td[1]').text
        # 사원수
        employee = driver.find_element(By.XPATH, '//*[@id="Contents"]/div[2]/div[1]/table/tbody/tr[3]/td[2]').text
        # 업종분류
        try:
            sector = driver.find_element(By.XPATH, '//*[@id="Contents"]/div[2]/div[1]/table/tbody/tr[5]/td[1]').text
        except:
            sector = driver.find_element(By.XPATH, '//*[@id="Contents"]/div[2]/div[1]/table/tbody/tr[4]/td[1]').text
        # 설립연도
        corp_year = driver.find_element(By.XPATH, '//*[@id="Contents"]/div[2]/div[1]/table/tbody/tr[1]/td[2]').text

        catch_temp = pd.DataFrame([corp_name, corp_type, sales_money, employee, sector, corp_year],
                                   index=['기업명','기업형태','매출액','사원수','업종분류','설립연도']).T
        catch_res = pd.concat([catch_res, catch_temp])

        time.sleep(0.2)
    except:
        pass
    
    driver.quit()

In [ ]:
catch_res.to_csv('./catch_result2.csv')  # 파일명만 변경하여 저장하시면 됩니다.

# 코드 수정

In [ ]:
import pandas as pd
# 정적크롤링
from bs4 import BeautifulSoup
import requests
import re
# 동적크롤링
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [ ]:
# 파일 불러오기: 불러올 파일을 현재 폴더에 넣어두세요.
df = pd.read_csv('./programmers.csv')  # 파일명만 변경하여 그대로 돌리시면 됩니다.
df_corp_name = df['기업명']  # 기업명 불러오기

In [ ]:
# 임시 프레임
catch_res = pd.DataFrame(columns=['기업명','기업형태','매출액','사원수','업종분류','설립연도'])

In [ ]:
# 크롤링 프로그램
for name in df_corp_name:
    url = f'https://www.catch.co.kr/Search/SearchDetail?Menu=2&Keyword={name}'  # 기업명으로 URL불러오기

    driver= webdriver.Chrome()
    driver.get(url)
    
    try:
        driver.find_element(By.XPATH, '//*[@id="leftTop"]/ul/li/div[2]/p[1]/a').click()
        time.sleep(0.2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 기업명
        corp_name = name
        corp_year = soup.select('td')[1].text  # 설립연도
        corp_type = soup.select('td')[2].text  # 기업 형태
        if len(soup.select('td')) == 10:
            sales_money = soup.select('td')[4].text  # 매출액
            employee = soup.select('td')[8].text  # 사원수
            sector = soup.select('td')[9].text  # 업종분류
        elif len(soup.select('td')) == 8:
            sales_money = soup.select('td')[4].text  # 매출액
            employee = soup.select('td')[5].text  # 사원수
            sector = soup.select('td')[6].text  # 업종분류        
        else:
            employee = soup.select('td')[4].text  # 사원수
            sector = soup.select('td')[5].text  # 업종분류

        catch_temp = pd.DataFrame([corp_name, corp_type, sales_money, employee, sector, corp_year],
                                   index=['기업명','기업형태','매출액','사원수','업종분류','설립연도']).T
        catch_res = pd.concat([catch_res, catch_temp])

        time.sleep(0.2)
    except:
        pass
    
driver.quit()

In [ ]:
catch_res.to_csv('./catch_result.csv')  # 파일명만 변경하여 저장하시면 됩니다.

# 2차 수정(최종본)
※ 주의사항: 사이트가 불안정하여 오류가 날 경우 time.sleep을 넉넉하게 설정하여 주세요

In [1]:
import pandas as pd
import numpy as np
# 정적크롤링
from bs4 import BeautifulSoup
import requests
import re
# 동적크롤링
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [2]:
corp_df = pd.read_csv('./catch_result.csv', index_col=0)  # 데이터 불러오기
df_res = pd.DataFrame(columns=['기업규모', '기업형태', '당기손익', '대표자', '매출액',
                                '사원수', '상세업종', '설립일', '신용등급', '영업이익', '카테고리'])  # 결과를 저장할 빈 프레임

In [3]:
# 크롤링 프로그램
for name in corp_df['기업명']:
    url = f'https://www.catch.co.kr/Search/SearchDetail?Menu=2&Keyword={name}'  # 기업명으로 URL불러오기

    driver= webdriver.Chrome()
    driver.get(url)

    driver.find_element(By.XPATH, '//*[@id="leftTop"]/ul/li/div[2]/p[1]/a').click()
    time.sleep(0.3)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    list = ['기업명']  # 컬럼이 될 리스트
    list2 = [name]  # 값이 될 리스트
    for i in soup.find_all('tbody')[0].select('th'):
        list.append(i.text)

    for j in soup.find_all('tbody')[0].select('td'):
        list2.append(j.text)

    df_temp = pd.DataFrame([list2], columns=list)
    df_temp.reset_index(drop=True, inplace=True)
    df_res = pd.concat([df_res, df_temp], ignore_index=True)  # 결과 DataFrame과 새로운 DataFrame을 결합
    df_res.reset_index(drop=True, inplace=True)

driver.quit()

In [5]:
df_res.columns  # 모든 컬럼 확인

Index(['기업규모', '기업형태', '당기손익', '대표자', '매출액', '사원수', '상세업종', '설립일', '신용등급',
       '영업이익', '카테고리', '기업명'],
      dtype='object')

In [7]:
df_res[['기업명','기업규모','매출액','사원수','설립일','상세업종']]  # 필요한 컬럼만 추출 전처리 필요함

,기업명,기업규모,매출액,사원수,설립일,상세업종
0,레몬베이스,중소기업,\n -,\n -명,2020년 10월 30일,응용 소프트웨어 개발 및 공급업
1,오토위니,대기업,"\n 연결 재무제표 : 2조 7,545억별도 재무제표 : 2조 2,408억\n","\n 4,672명 (2023.09)",2000년 04월 10일,응용 소프트웨어 개발 및 공급업
2,페이타랩,중소기업,\n 5억 (2021.12),\n 52명 (2022.07),2018년 02월 28일,시스템 소프트웨어 개발 및 공급업
3,한화비전,대기업,"\n 8,041억 (2022.12)",\n 781명 (2022.12),2018년 04월 02일,방송장비 제조업
4,발란,중소기업,\n 891억 (2022.12),\n 122명 (2022.12),2015년 05월 14일,전자상거래 소매 중개업
5,플라잉캣,중소기업,NaN,\n 18명 (2022.10),2018년 02월 27일,기타 통신 판매업
6,AB180,중소기업,\n 272억 (2022.12),\n -명,2015년 11월 04일,응용 소프트웨어 개발 및 공급업
7,월급쟁이부자들,중소기업,\n 118억 (2021.12),\n -명,2018년 03월 27일,기타 기술 및 직업 훈련학원
8,에잇퍼센트,중소기업,\n 31억 (2022.12),\n 55명 (2022.12),2014년 11월 13일,시스템 소프트웨어 개발 및 공급업
9,앤트,중견기업,\n 194억 (2022.12),\n 103명 (2023.04),1996년 01월 16일,응용 소프트웨어 개발 및 공급업
